# Pokemon Ranks

This notebook is used to aggregate the results from the simulation and to compute the ranks.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/results/simulation_stats.csv')

In [3]:
df.head()

,pokemon,pokemonb,avg_moves,pokemon_wins,pokemonb_wins,ties
0,aerodactyl,porygon,8.382,1000,0,0
1,dragonair,porygon,10.319,136,864,0
2,nidorino,porygon,6.935,21,979,0
3,golbat,porygon,10.092,396,604,0
4,electabuzz,porygon,8.715,892,108,0


In [4]:
pokemon_stats_df = pd.read_csv('../data/processed/pokemon_stats.csv')

# Top 10 Ties

In [5]:
df.sort_values('ties', ascending=False).head(10)

,pokemon,pokemonb,avg_moves,pokemon_wins,pokemonb_wins,ties
3288,chansey,gastly,122.000,0,0,1000
3271,chansey,haunter,121.267,0,17,983
3332,chansey,gengar,107.986,0,362,638
2980,onix,wigglytuff,33.420,503,0,497
5879,clefable,onix,35.200,0,508,492
2919,gastly,wigglytuff,28.392,566,0,434
5818,clefable,gastly,26.252,0,588,412
5822,clefable,slowpoke,25.047,166,434,400
2902,haunter,wigglytuff,26.069,626,0,374
5875,clefable,muk,22.796,2,629,369


# Aggregate Total Wins, Losses, Ties

In [6]:
stats = {}

def aggregate_results(row, is_a=True):
    label = 'pokemon'
    other_label = 'poke'
    if not is_a:
        label = 'pokemonb'
    
    if row[label] not in stats:
        stats[row[label]] = {
            'wins': 0,
            'losses': 0,
            'ties': 0
        }
    
    wins = row['{}_wins'.format(label)]
    ties = row['ties']
    stats[row[label]]['wins'] += row['{}_wins'.format(label)]
    stats[row[label]]['ties'] += row['ties']
    stats[row[label]]['losses'] += 1000 - wins - ties

for _, row in df.iterrows():
    aggregate_results(row)
    aggregate_results(row, is_a=False)

In [7]:
stats_df = pd.DataFrame(stats).transpose()
stats_df.index.name = 'Pokemon'

In [8]:
stats_df['matches'] = stats_df['wins'] + stats_df['losses'] + stats_df['ties']

## Compute Rank

Rank is computed in two ways:

1. Simple frequentist approach (wins / total number of battles)
2. Bayesian approach - beta distribution

In [9]:
from scipy.stats import beta

In [10]:
stats_df['% win'] = (stats_df['wins'] / stats_df['matches']) * 100
stats_df['% loss'] = (stats_df['losses'] / stats_df['matches']) * 100
stats_df['% tie'] = (stats_df['ties'] / stats_df['matches']) * 100

a0 = 0
b0 = 0
stats_df['rank'] = beta.ppf(0.001, stats_df['wins'] + a0, stats_df['losses'] + b0)

In [11]:
stats_df = pd.merge(stats_df, pokemon_stats_df, left_index=True, right_on='pokemon', how='inner')

In [12]:
stats_df = stats_df.set_index('pokemon')

In [13]:
stats_df.head()

,wins,losses,ties,matches,% win,% loss,% tie,rank,attack,defense,hp,special-attack,special-defense,speed,types
pokemon,,,,,,,,,,,,,,,
aerodactyl,121533,24467,0,146000,83.241781,16.758219,0.000000,0.829384,105,65,80,60,75,130,"flying,rock"
porygon,85104,60891,5,146000,58.290411,41.706164,0.003425,0.578933,60,70,65,85,75,40,normal
dragonair,67418,78574,8,146000,46.176712,53.817808,0.005479,0.457762,84,65,61,70,70,70,dragon
nidorino,40401,105599,0,146000,27.671918,72.328082,0.000000,0.273110,72,57,61,55,55,65,poison
golbat,97828,48172,0,146000,67.005479,32.994521,0.000000,0.666246,80,70,75,65,75,90,"flying,poison"


## Pokemon Rank (Best to Worst)

In [14]:
with pd.option_context("display.max_rows", 1000):
    display(stats_df.sort_values('rank', ascending=False))

,wins,losses,ties,matches,% win,% loss,% tie,rank,attack,defense,hp,special-attack,special-defense,speed,types
pokemon,,,,,,,,,,,,,,,
mewtwo,143219,2781,0,146000,98.095205,1.904795,0.000000,0.979828,110,90,106,154,90,130,psychic
snorlax,140360,5566,74,146000,96.136986,3.812329,0.050685,0.960290,110,65,160,65,110,30,normal
lapras,138327,7673,0,146000,94.744521,5.255479,0.000000,0.945623,85,80,130,85,95,60,"ice,water"
articuno,137096,8904,0,146000,93.901370,6.098630,0.000000,0.937061,85,100,90,95,125,85,"flying,ice"
mew,136855,9145,0,146000,93.736301,6.263699,0.000000,0.935386,100,100,100,100,100,100,psychic
zapdos,133370,12630,0,146000,91.349315,8.650685,0.000000,0.911204,90,85,90,125,90,100,"flying,electric"
slowbro,132887,13113,0,146000,91.018493,8.981507,0.000000,0.907857,75,110,95,100,80,30,"psychic,water"
moltres,130264,15736,0,146000,89.221918,10.778082,0.000000,0.889696,100,90,90,125,85,90,"flying,fire"
golem,126347,19653,0,146000,86.539041,13.460959,0.000000,0.862616,120,130,80,55,65,45,"ground,rock"


In [15]:
stats_df.sort_values('rank', ascending=False).to_csv('../data/results/pokemon_ranks.csv')